# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7f241b8f70>
├── 'a' --> tensor([[ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431]])
└── 'x' --> <FastTreeValue 0x7f7f241b8d30>
    └── 'c' --> tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                        [ 0.0976, -2.0761,  0.4639,  0.2172],
                        [ 1.6092,  0.8168, -1.7655, -0.6879]])

In [4]:
t.a

tensor([[ 0.8769, -0.4029, -0.0827],
        [ 0.3252, -0.1304,  0.5431]])

In [5]:
%timeit t.a

79.5 ns ± 0.797 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7f241b8f70>
├── 'a' --> tensor([[-0.8736,  0.2295, -0.1274],
│                   [-0.2980,  0.5142,  0.1896]])
└── 'x' --> <FastTreeValue 0x7f7f241b8d30>
    └── 'c' --> tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                        [ 0.0976, -2.0761,  0.4639,  0.2172],
                        [ 1.6092,  0.8168, -1.7655, -0.6879]])

In [7]:
%timeit t.a = new_value

90.9 ns ± 0.699 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8769, -0.4029, -0.0827],
               [ 0.3252, -0.1304,  0.5431]]),
    x: Batch(
           c: tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8769, -0.4029, -0.0827],
        [ 0.3252, -0.1304,  0.5431]])

In [11]:
%timeit b.a

76 ns ± 1.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4494, -0.5239, -1.8917],
               [-0.2111,  0.1991, -0.4237]]),
    x: Batch(
           c: tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879]]),
       ),
)

In [13]:
%timeit b.a = new_value

687 ns ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.1 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

274 µs ± 8.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

267 µs ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7e6e41ff40>
├── 'a' --> tensor([[[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]],
│           
│                   [[ 0.8769, -0.4029, -0.0827],
│                    [ 0.3252, -0.1304,  0.5431]]])
└── 'x' --> <FastTreeValue 0x7f7e6e3f88b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.3 µs ± 552 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7e6e408040>
├── 'a' --> tensor([[ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431],
│                   [ 0.8769, -0.4029, -0.0827],
│                   [ 0.3252, -0.1304,  0.5431]])
└── 'x' --> <FastTreeValue 0x7f7e6f2a1460>
    └── 'c' --> tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                        [ 0.0976, -2.0761,  0.4639,  0.2172],
                 

In [23]:
%timeit t_cat(trees)

43.5 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84.7 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7474,  1.3314, -1.4700, -0.6149],
                       [ 0.0976, -2.0761,  0.4639,  0.2172],
                       [ 1.6092,  0.8168, -1.7655, -0.6879]],
              
                      [[ 0.7474,  1.3314, -1.4700, -0.6149],
                       [ 0.0976, -2.0761,  0.4639,  0.2172],
                       [ 1.6092,  0.8168, -1.7655, -0.6879]],
              
                      [[ 0.7474,  1.3314, -1.4700, -0.6149],
                       [ 0.0976, -2.0761,  0.4639,  0.2172],
                       [ 1.6092,  0.8168, -1.7655, -0.6879]],
              
                      [[ 0.7474,  1.3314, -1.4700, -0.6149],
                       [ 0.0976, -2.0761,  0.4639,  0.2172],
                       [ 1.6092,  0.8168, -1.7655, -0.6879]],
              
                      [[ 0.7474,  1.3314, -1.4700, -0.6149],
                       [ 0.0976, -2.0761,  0.4639,  0.2172],
                       [ 1.6092,  0.8168, -1.7655, -0.6879]],

In [26]:
%timeit Batch.stack(batches)

111 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879],
                      [ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879],
                      [ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879],
                      [ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879],
                      [ 0.7474,  1.3314, -1.4700, -0.6149],
                      [ 0.0976, -2.0761,  0.4639,  0.2172],
                      [ 1.6092,  0.8168, -1.7655, -0.6879],
                      [ 0.7474,  1.3314, -1.4700, -0.6149],
                   

In [28]:
%timeit Batch.cat(batches)

197 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

578 µs ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
